In [ ]:
import numpy as np
import pandas as pd
import umap
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import plotly.graph_objects as go
from scipy.spatial import procrustes

# ==========================================================
# ⚙️ 操作可能パラメータ (Parameters for User Control)
# ==========================================================
# 1. デフォルトのクラスタ数 (KMeansでデータを分類する際の目標クラスタ数)
N_CLUSTERS_DEFAULT = 20

# 2. サブセットとして選択するクラスタIDのリスト
#    このリストに含まれるクラスタIDが再埋め込みの対象になります。
SELECTED_CLUSTER_IDS = [0, 1, 2, 3, 4, 5]  # 例: 最初の6クラスタを選択
# ==========================================================
# ⚠️ 注意: 実際のクラスタIDはデータ生成とKMeansの結果によって決定されます。
#    N_CLUSTERS_DEFAULT に近い範囲のIDを指定してください。
# ==========================================================


# --- 1. データ生成 ---
n_samples = 1000
n_features = 10
random_state = 42

# データを生成
X_a, y_a = make_blobs(n_samples=n_samples // 2, centers=N_CLUSTERS_DEFAULT // 2, n_features=n_features, random_state=random_state)
X_b, y_b = make_blobs(n_samples=n_samples // 2, centers=N_CLUSTERS_DEFAULT // 2, n_features=n_features, random_state=random_state + 1, cluster_std=2.0)
y_b = y_b + N_CLUSTERS_DEFAULT // 2 # クラスタIDをずらす

X = np.vstack((X_a, X_b))
print(f"Generated data shape: {X.shape}")

# --- 2. UMAP 埋め込み（初期グローバル） ---
print("Performing initial UMAP embedding...")
reducer_global = umap.UMAP(random_state=random_state, n_components=2)
embedding_global = reducer_global.fit_transform(X)

df = pd.DataFrame(X)
df['global_x'] = embedding_global[:, 0]
df['global_y'] = embedding_global[:, 1]

# --- 3. クラスタリングと色付け (N_CLUSTERS_DEFAULTを使用) ---
print(f"Performing KMeans clustering with {N_CLUSTERS_DEFAULT} clusters...")
kmeans = KMeans(n_clusters=N_CLUSTERS_DEFAULT, random_state=random_state, n_init='auto')
df['cluster_id'] = kmeans.fit_predict(X)

# クラスタIDに対応する色を決定
colors = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5',
    '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5'
]
unique_clusters = sorted(df['cluster_id'].unique())
cluster_colors = {cluster: colors[i % len(colors)] for i, cluster in enumerate(unique_clusters)}
df['color'] = df['cluster_id'].map(cluster_colors)
df['cluster_id'] = df['cluster_id'].astype(str) # Plotlyの表示用に文字列に変換

# --- 4. サブセットの選択 (SELECTED_CLUSTER_IDSを使用) ---
df['is_selected'] = df['cluster_id'].isin([str(cid) for cid in SELECTED_CLUSTER_IDS]) # 選択リストを文字列に変換
df_selected = df[df['is_selected']].copy()

if df_selected.empty:
    print("⚠️ 警告: 選択されたクラスタIDのリストに該当するデータポイントがありません。アニメーションは実行されません。")
    # 代替として全データで続行
    df_selected = df.copy()
    df_selected['is_selected'] = True
    SELECTED_CLUSTER_IDS_STR = ["All Data"]
else:
    SELECTED_CLUSTER_IDS_STR = [str(cid) for cid in SELECTED_CLUSTER_IDS]

print(f"Selected {len(df_selected)} data points from clusters: {', '.join(SELECTED_CLUSTER_IDS_STR)}")

# --- 5. サブセット UMAP 埋め込み ---
print("Performing UMAP embedding for selected subset...")
reducer_subset = umap.UMAP(random_state=random_state + 2, n_components=2)
# UMAP入力データは、特徴量ベクトルのみ
embedding_subset = reducer_subset.fit_transform(df_selected.iloc[:, :n_features].values) 

df_selected['subset_x'] = embedding_subset[:, 0]
df_selected['subset_y'] = embedding_subset[:, 1]

# --- 6. アニメーションのためのデータ準備 (Procrustes アライメント) ---
old_coords = df_selected[['global_x', 'global_y']].values
new_coords = df_selected[['subset_x', 'subset_y']].values

# Procrustes Analysisを実行し、新しい座標を古い座標にアライメントする
# mtx1, mtx2がアライメント後の座標。mtx2が新しい座標をアライメントしたもの
mtx1, mtx2, disparity = procrustes(old_coords, new_coords) 
df_selected['aligned_subset_x'] = mtx2[:, 0]
df_selected['aligned_subset_y'] = mtx2[:, 1]

# dfにsubset_x, subset_y, aligned_subset_x, aligned_subset_yをマージ
# ⭐️ 修正点: インデックス結合 (left_index=True, right_index=True) を使用
df = df.merge(
    df_selected[['subset_x', 'subset_y', 'aligned_subset_x', 'aligned_subset_y']],
    left_index=True,  # 左側のDataFrame (df) のインデックスを使用
    right_index=True, # 右側のDataFrame (df_selected) のインデックスを使用
    how='left',
    suffixes=('', '_new')
)

# ⭐️ 警告対策: マージ後のNaNを0で埋める（非選択データはアニメーションに影響しないが、補間計算がエラーにならないように）
# 実際には、非選択の点はアニメーション中に current_x, current_y が変化しないため、この処理は必須ではないが、
# 互換性のないdtypeの警告対策として、NaNを含む列を浮動小数点型として扱うために、以下で初期化する。
df['aligned_subset_x'] = df['aligned_subset_x'].fillna(df['global_x'])
df['aligned_subset_y'] = df['aligned_subset_y'].fillna(df['global_y'])
df['subset_x'] = df['subset_x'].fillna(df['global_x'])
df['subset_y'] = df['subset_y'].fillna(df['global_y'])

# --- 7. Plotly Scatter デモ ---

# フレームデータを作成するヘルパー関数
def create_frame_data(df_frame, opacity_selected, opacity_unselected, x_col, y_col, title_suffix=""):
    traces = []
    # 非選択データ（背景として薄く表示）
    traces.append(go.Scatter(
        x=df_frame[~df_frame['is_selected']][x_col],
        y=df_frame[~df_frame['is_selected']][y_col],
        mode='markers',
        marker=dict(color='lightgray', size=5, opacity=opacity_unselected),
        name='Unselected',
        showlegend=False,
        hoverinfo='skip'
    ))
    # 選択データ
    for cluster_id_str in unique_clusters:
        df_cluster = df_frame[df_frame['cluster_id'] == cluster_id_str]
        if not df_cluster[df_cluster['is_selected']].empty: # 選択されたデータのみ表示
            traces.append(go.Scatter(
                x=df_cluster[df_cluster['is_selected']][x_col],
                y=df_cluster[df_cluster['is_selected']][y_col],
                mode='markers',
                marker=dict(color=cluster_colors[int(cluster_id_str)], size=8, opacity=opacity_selected),
                name=f'Cluster {cluster_id_str}',
                showlegend=True,
                hovertemplate="<b>Cluster ID:</b> %{customdata[0]}<br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>",
                customdata=df_cluster[df_cluster['is_selected']][['cluster_id']]
            ))
    return traces, f"UMAP Visualization - {title_suffix}"

# アニメーションフレームの生成
frames_aligned = []
frames_random = []
num_frames = 60
animation_duration = 1000 # ms

for i in range(num_frames):
    t = i / (num_frames - 1) # 0から1へ線形補間

    # 選択点の座標を補間
    df_interp_aligned = df.copy()
    # 選択された点のみ、global -> aligned_subset へ補間
    df_interp_aligned.loc[df['is_selected'], 'current_x'] = df.loc[df['is_selected'], 'global_x'] * (1 - t) + df.loc[df['is_selected'], 'aligned_subset_x'] * t
    df_interp_aligned.loc[df['is_selected'], 'current_y'] = df.loc[df['is_selected'], 'global_y'] * (1 - t) + df.loc[df['is_selected'], 'aligned_subset_y'] * t
    
    df_interp_random = df.copy()
    # 選択された点のみ、global -> subset へ補間 (アライメントなし)
    df_interp_random.loc[df['is_selected'], 'current_x'] = df.loc[df['is_selected'], 'global_x'] * (1 - t) + df.loc[df['is_selected'], 'subset_x'] * t
    df_interp_random.loc[df['is_selected'], 'current_y'] = df.loc[df['is_selected'], 'global_y'] * (1 - t) + df.loc[df['is_selected'], 'subset_y'] * t

    # 非選択点の透明度を調整 (アニメーションの進行とともに薄く)
    opacity_unselected = 1.0 - t * 0.8
    opacity_selected = 1.0

    traces_aligned, title_aligned = create_frame_data(df_interp_aligned, opacity_selected, opacity_unselected, 'current_x', 'current_y', "Procrustes Aligned")
    frames_aligned.append(go.Frame(data=traces_aligned, name=str(i) + "_a"))

    traces_random, title_random = create_frame_data(df_interp_random, opacity_selected, opacity_unselected, 'current_x', 'current_y', "No Alignment (Random Init)")
    frames_random.append(go.Frame(data=traces_random, name=str(i) + "_r"))


# 初期描画（グローバルビュー）
initial_traces_aligned, initial_title_aligned = create_frame_data(df, 1.0, 1.0, 'global_x', 'global_y', f"Procrustes Aligned (Selected: {', '.join(SELECTED_CLUSTER_IDS_STR)})")
initial_traces_random, initial_title_random = create_frame_data(df, 1.0, 1.0, 'global_x', 'global_y', f"No Alignment (Selected: {', '.join(SELECTED_CLUSTER_IDS_STR)})")


# レイアウト設定
fig = go.Figure(
    data=initial_traces_aligned + initial_traces_random,
    layout=go.Layout(
        title_text=f"UMAP Zoom Animation Demo (KMeans Clusters: {N_CLUSTERS_DEFAULT})",
        hovermode="closest",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": animation_duration // num_frames, "redraw": True},
                                        "fromcurrent": True, "transition": {"duration": 0, "easing": "linear"}}])],
            x=0, xanchor="left", y=1.1, yanchor="top"
        )],
        sliders=[dict(
            steps=[dict(method='animate', args=[[f.name], {"mode": "immediate", "frame": {"duration": animation_duration // num_frames, "redraw": True}, "transition": {"duration": 0}}], label=str(k)) for k, f in enumerate(frames_aligned)],
            active=0,
            transition={"duration": 0},
            x=0, xanchor="left", y=0, yanchor="top",
            currentvalue=dict(font=dict(size=12), prefix="Frame: ", visible=True, xanchor="right")
        )],
        annotations=[
            dict(text=initial_title_aligned, x=0.25, y=1.05, xref="paper", yref="paper", font=dict(size=14)),
            dict(text=initial_title_random, x=0.75, y=1.05, xref="paper", yref="paper", font=dict(size=14))
        ]
    ),
    frames=[] # 初期は空、結合したフレームを後で設定
)

# サブプロットの作成
fig.update_layout(
    xaxis=dict(domain=[0, 0.48], showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(domain=[0, 1], showgrid=False, zeroline=False, showticklabels=False),
    xaxis2=dict(domain=[0.52, 1], showgrid=False, zeroline=False, showticklabels=False),
    yaxis2=dict(domain=[0, 1], showgrid=False, zeroline=False, showticklabels=False),
    legend=dict(x=1.05, y=1, xanchor='left', yanchor='top'),
    height=600
)

# トレースをそれぞれのサブプロットに割り当てる (初期描画)
# Procrustes Aligned Plot (左)
for i, trace in enumerate(fig.data[:len(initial_traces_aligned)]):
    trace.update(xaxis='x', yaxis='y')

# No Alignment Plot (右)
for i, trace in enumerate(fig.data[len(initial_traces_aligned):]):
    trace.update(xaxis='x2', yaxis='y2')


# フレーム内のデータもサブプロットに割り当てるために、フレームを結合
combined_frames = []
for frame_idx in range(len(frames_aligned)):
    # 左右のフレームデータを結合
    combined_frame_data = frames_aligned[frame_idx].data + frames_random[frame_idx].data

    # Aligned Frames (左)
    for i, trace in enumerate(combined_frame_data[:len(initial_traces_aligned)]):
        trace.update(xaxis='x', yaxis='y')
    # Random Frames (右)
    for i, trace in enumerate(combined_frame_data[len(initial_traces_aligned):]):
        trace.update(xaxis='x2', yaxis='y2')
        
    combined_frames.append(go.Frame(data=combined_frame_data, name=frames_aligned[frame_idx].name)) # 名前は片方を使用

fig.frames = combined_frames
fig.update_layout(height=800, width=1200)

fig.show()

In [14]:
import numpy as np
import pandas as pd
import umap
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import plotly.graph_objects as go
from scipy.spatial import procrustes # 今回は使用しないが、念のためインポートは残す

# ==========================================================
# ⚙️ 操作可能パラメータ (Parameters for User Control)
# ==========================================================
# 1. デフォルトのクラスタ数 (KMeansでデータを分類する際の目標クラスタ数)
N_CLUSTERS_DEFAULT = 30

# 2. サブセットとして選択するクラスタIDのリスト
#    このリストに含まれるクラスタIDが再埋め込みの対象になります。
SELECTED_CLUSTER_IDS = [0, 1, 2, 3, 4, 5]  # 例: 最初の6クラスタを選択
# ==========================================================


# --- 1. データ生成 ---
n_samples = 1000
n_features = 10
random_state = 42

# データを生成
X_a, y_a = make_blobs(n_samples=n_samples // 2, centers=N_CLUSTERS_DEFAULT // 2, n_features=n_features, random_state=random_state)
X_b, y_b = make_blobs(n_samples=n_samples // 2, centers=N_CLUSTERS_DEFAULT // 2, n_features=n_features, random_state=random_state + 1, cluster_std=2.0)

X = np.vstack((X_a, X_b))
print(f"Generated data shape: {X.shape}")

# --- 2. UMAP 埋め込み（初期グローバル） ---
print("Performing initial UMAP embedding...")
reducer_global = umap.UMAP(random_state=random_state, n_components=2)
embedding_global = reducer_global.fit_transform(X)

df = pd.DataFrame(X)
df['global_x'] = embedding_global[:, 0]
df['global_y'] = embedding_global[:, 1]

# --- 3. クラスタリングと色付け (N_CLUSTERS_DEFAULTを使用) ---
print(f"Performing KMeans clustering with {N_CLUSTERS_DEFAULT} clusters...")
kmeans = KMeans(n_clusters=N_CLUSTERS_DEFAULT, random_state=random_state, n_init='auto')
df['cluster_id_int'] = kmeans.fit_predict(X) 

# クラスタIDに対応する色を決定
colors = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5',
    '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5'
]
unique_clusters = sorted(df['cluster_id_int'].unique())
cluster_colors = {cluster: colors[i % len(colors)] for i, cluster in enumerate(unique_clusters)}

# Plotlyの表示および比較用に文字列型に変換
df['cluster_id'] = df['cluster_id_int'].astype(str)

# --- 4. サブセットの選択 (SELECTED_CLUSTER_IDSを使用) ---
df['is_selected'] = df['cluster_id_int'].isin(SELECTED_CLUSTER_IDS) 
df_selected = df[df['is_selected']].copy()

if df_selected.empty:
    print("⚠️ 警告: 選択されたクラスタIDのリストに該当するデータポイントがありません。アニメーションは実行されません。")
    df_selected = df.copy()
    df_selected['is_selected'] = True
    SELECTED_CLUSTER_IDS_STR = ["All Data"]
else:
    SELECTED_CLUSTER_IDS_STR = [str(cid) for cid in SELECTED_CLUSTER_IDS]

print(f"Selected {len(df_selected)} data points from clusters: {', '.join(SELECTED_CLUSTER_IDS_STR)}")

# --- 5. サブセット UMAP 埋め込み (UMAP initを使用) ---
print("Performing UMAP embedding for selected subset using global coordinates as initialization...")

# 🌟 修正点 A: 初期座標として、選択された点のグローバル座標を抽出
initial_coords = df_selected[['global_x', 'global_y']].values 

reducer_subset_aligned = umap.UMAP(
    random_state=random_state + 2, 
    n_components=2,
    init=initial_coords,  # メンタルマップ維持のため、グローバル座標を初期配置として使用
    n_epochs=200
)

# メンタルマップ維持版の座標
embedding_aligned = reducer_subset_aligned.fit_transform(df_selected.iloc[:, :n_features].values) 
df_selected['aligned_subset_x'] = embedding_aligned[:, 0]
df_selected['aligned_subset_y'] = embedding_aligned[:, 1]

# 比較用 (非アライメント/ランダム初期化版) の座標
reducer_subset_random = umap.UMAP(
    random_state=random_state + 3, 
    n_components=2,
    init='spectral',  # デフォルトの初期化 (ランダムな結果になりやすい)
    n_epochs=200
)
embedding_random = reducer_subset_random.fit_transform(df_selected.iloc[:, :n_features].values)
df_selected['subset_x'] = embedding_random[:, 0]
df_selected['subset_y'] = embedding_random[:, 1]


# --- 6. データフレームへのマージ (Procrustes処理は不要) ---

# dfに座標をマージ (インデックス結合を使用)
df = df.merge(
    df_selected[['subset_x', 'subset_y', 'aligned_subset_x', 'aligned_subset_y']],
    left_index=True,  
    right_index=True, 
    how='left',
    suffixes=('', '_new')
)

# 欠損値をグローバル座標で埋める (非選択の点は動かないため)
df['aligned_subset_x'] = df['aligned_subset_x'].fillna(df['global_x'])
df['aligned_subset_y'] = df['aligned_subset_y'].fillna(df['global_y'])
df['subset_x'] = df['subset_x'].fillna(df['global_x'])
df['subset_y'] = df['subset_y'].fillna(df['global_y'])


# --- 7. Plotly Scatter デモ ---

# 軸範囲の計算 (すべてのグローバル座標に基づく)
all_x = df['global_x'].values
all_y = df['global_y'].values
min_x, max_x = np.min(all_x), np.max(all_x)
min_y, max_y = np.min(all_y), np.max(all_y)
padding = 0.05
x_range = [min_x - (max_x - min_x) * padding, max_x + (max_x - min_x) * padding]
y_range = [min_y - (max_y - min_y) * padding, max_y + (max_y - min_y) * padding]


# フレームデータを作成するヘルパー関数 (変更なし)
def create_frame_data(df_frame, opacity_selected, opacity_unselected, x_col, y_col, title_suffix=""):
    traces = []
    
    # 1. 非選択データ（背景として薄く表示）
    traces.append(go.Scatter(
        x=df_frame[~df_frame['is_selected']][x_col],
        y=df_frame[~df_frame['is_selected']][y_col],
        mode='markers',
        marker=dict(color='lightgray', size=5, opacity=opacity_unselected),
        name='Unselected',
        showlegend=False,
        hoverinfo='skip'
    ))
    
    # 2. 選択データ (クラスタごとに色分けして表示)
    for cluster_id_int in unique_clusters: 
        cluster_id_str = str(cluster_id_int)
        
        df_cluster = df_frame[df_frame['cluster_id'] == cluster_id_str] 
        df_selected_cluster = df_cluster[df_cluster['is_selected']]

        if not df_selected_cluster.empty: 
            traces.append(go.Scatter(
                x=df_selected_cluster[x_col],
                y=df_selected_cluster[y_col],
                mode='markers',
                marker=dict(color=cluster_colors[cluster_id_int], size=8, opacity=opacity_selected),
                name=f'Cluster {cluster_id_str}',
                showlegend=True,
                hovertemplate="<b>Cluster ID:</b> %{customdata[0]}<br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>",
                customdata=df_selected_cluster[['cluster_id']]
            ))
    return traces, f"UMAP Visualization - {title_suffix}"

# アニメーションフレームの生成
frames_aligned = []
frames_random = []
num_frames = 60
animation_duration = 1000 

for i in range(num_frames):
    t = i / (num_frames - 1)

    # UMAP init使用版 (aligned_subset_x/y) への補間
    df_interp_aligned = df.copy()
    df_interp_aligned.loc[df['is_selected'], 'current_x'] = df.loc[df['is_selected'], 'global_x'] * (1 - t) + df.loc[df['is_selected'], 'aligned_subset_x'] * t
    df_interp_aligned.loc[df['is_selected'], 'current_y'] = df.loc[df['is_selected'], 'global_y'] * (1 - t) + df.loc[df['is_selected'], 'aligned_subset_y'] * t
    
    # デフォルトinit使用版 (subset_x/y) への補間
    df_interp_random = df.copy()
    df_interp_random.loc[df['is_selected'], 'current_x'] = df.loc[df['is_selected'], 'global_x'] * (1 - t) + df.loc[df['is_selected'], 'subset_x'] * t
    df_interp_random.loc[df['is_selected'], 'current_y'] = df.loc[df['is_selected'], 'global_y'] * (1 - t) + df.loc[df['is_selected'], 'subset_y'] * t

    opacity_unselected = 1.0 - t * 0.8
    opacity_selected = 1.0

    traces_aligned, title_aligned = create_frame_data(df_interp_aligned, opacity_selected, opacity_unselected, 'current_x', 'current_y', "UMAP Init (Global Coords)")
    frames_aligned.append(go.Frame(data=traces_aligned, name=str(i) + "_a"))

    traces_random, title_random = create_frame_data(df_interp_random, opacity_selected, opacity_unselected, 'current_x', 'current_y', "UMAP Default Init (Spectral)")
    frames_random.append(go.Frame(data=traces_random, name=str(i) + "_r"))


# 初期描画（グローバルビュー）
initial_traces_aligned, initial_title_aligned = create_frame_data(df, 1.0, 1.0, 'global_x', 'global_y', f"UMAP Init (Selected: {', '.join(SELECTED_CLUSTER_IDS_STR)})")
initial_traces_random, initial_title_random = create_frame_data(df, 1.0, 1.0, 'global_x', 'global_y', f"UMAP Default Init (Selected: {', '.join(SELECTED_CLUSTER_IDS_STR)})")


# レイアウト設定
fig = go.Figure(
    data=initial_traces_aligned + initial_traces_random,
    layout=go.Layout(
        title_text=f"UMAP Zoom Animation Demo (KMeans Clusters: {N_CLUSTERS_DEFAULT})",
        hovermode="closest",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": animation_duration // num_frames, "redraw": True},
                                        "fromcurrent": True, "transition": {"duration": 0, "easing": "linear"}}])],
            x=0, xanchor="left", y=1.1, yanchor="top"
        )],
        sliders=[dict(
            steps=[dict(method='animate', args=[[f.name], {"mode": "immediate", "frame": {"duration": animation_duration // num_frames, "redraw": True}, "transition": {"duration": 0}}], label=str(k)) for k, f in enumerate(frames_aligned)],
            active=0,
            transition={"duration": 0},
            x=0, xanchor="left", y=0, yanchor="top",
            currentvalue=dict(font=dict(size=12), prefix="Frame: ", visible=True, xanchor="right")
        )],
        annotations=[
            dict(text=initial_title_aligned, x=0.25, y=1.05, xref="paper", yref="paper", font=dict(size=14)),
            dict(text=initial_title_random, x=0.75, y=1.05, xref="paper", yref="paper", font=dict(size=14))
        ]
    ),
    frames=[]
)

# 統一された軸範囲とアスペクト比を設定
fig.update_layout(
    # 左側のプロット (x, y軸)
    xaxis=dict(domain=[0, 0.48], showgrid=False, zeroline=False, showticklabels=False, 
               range=x_range, scaleanchor='y', scaleratio=1),
    yaxis=dict(domain=[0, 1], showgrid=False, zeroline=False, showticklabels=False, 
               range=y_range, scaleanchor='x', scaleratio=1),

    # 右側のプロット (x2, y2軸)
    xaxis2=dict(domain=[0.52, 1], showgrid=False, zeroline=False, showticklabels=False, 
                range=x_range, scaleanchor='y2', scaleratio=1),
    yaxis2=dict(domain=[0, 1], showgrid=False, zeroline=False, showticklabels=False, 
                range=y_range, scaleanchor='x2', scaleratio=1),
                
    legend=dict(x=1.05, y=1, xanchor='left', yanchor='top'),
    height=800, 
    width=1200
)

# トレースをそれぞれのサブプロットに割り当てる
for i, trace in enumerate(fig.data[:len(initial_traces_aligned)]):
    trace.update(xaxis='x', yaxis='y')

for i, trace in enumerate(fig.data[len(initial_traces_aligned):]):
    trace.update(xaxis='x2', yaxis='y2')


# フレーム内のデータもサブプロットに割り当てるために、フレームを結合
combined_frames = []
for frame_idx in range(len(frames_aligned)):
    combined_frame_data = frames_aligned[frame_idx].data + frames_random[frame_idx].data

    for i, trace in enumerate(combined_frame_data[:len(initial_traces_aligned)]):
        trace.update(xaxis='x', yaxis='y')

    for i, trace in enumerate(combined_frame_data[len(initial_traces_aligned):]):
        trace.update(xaxis='x2', yaxis='y2')
        
    combined_frames.append(go.Frame(data=combined_frame_data, name=frames_aligned[frame_idx].name))

fig.frames = combined_frames

fig.show()

Generated data shape: (1000, 10)
Performing initial UMAP embedding...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



Performing KMeans clustering with 30 clusters...
Selected 180 data points from clusters: 0, 1, 2, 3, 4, 5
Performing UMAP embedding for selected subset using global coordinates as initialization...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [13]:
# --- 座標 Min/Max 表示コード (UMAP init使用) ---

# 1. 次元削減前（特徴量ベクトル）の Min/Max
features = df.iloc[:, :n_features].values
print("## 📏 次元削減前 (特徴量ベクトル) の Min/Max")
print(f"Min: {np.min(features):.4f}")
print(f"Max: {np.max(features):.4f}")
print("---")

# 2. 次元削減後（グローバル座標）の Min/Max
print("## 🗺️ グローバル UMAP 座標 (global_x / global_y) の Min/Max")
print(f"X軸 Min: {df['global_x'].min():.4f}, Max: {df['global_x'].max():.4f}")
print(f"Y軸 Min: {df['global_y'].min():.4f}, Max: {df['global_y'].max():.4f}")
print("---")

# 3. 🔬 UMAP init を使用したサブセット座標の Min/Max
df_subset_coords = df[df['is_selected']]
print("## 🔬 UMAP init を使用したサブセット座標 (subset_x / subset_y) の Min/Max")
# 'subset_x'と'subset_y'には、初期座標をグローバル座標としたUMAPの結果が入る
print(f"X軸 Min: {df_subset_coords['subset_x'].min():.4f}, Max: {df_subset_coords['subset_x'].max():.4f}")
print(f"Y軸 Min: {df_subset_coords['subset_y'].min():.4f}, Max: {df_subset_coords['subset_y'].max():.4f}")
print("---")

## 📏 次元削減前 (特徴量ベクトル) の Min/Max
Min: -15.7621
Max: 15.4856
---
## 🗺️ グローバル UMAP 座標 (global_x / global_y) の Min/Max
X軸 Min: -11.7270, Max: 16.8161
Y軸 Min: -14.7733, Max: 22.1142
---
## 🔬 UMAP init を使用したサブセット座標 (subset_x / subset_y) の Min/Max
X軸 Min: -1.8870, Max: 16.6955
Y軸 Min: -5.1548, Max: 12.8434
---
